In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 10)

In [43]:
order = pd.read_excel("../附件1 近5年402家供应商的相关数据.xlsx", sheet_name=0, index_col=0)
supply = pd.read_excel("../附件1 近5年402家供应商的相关数据.xlsx", sheet_name=1, index_col=0)
display(order.head(), supply.head())

,材料分类,W001,W002,W003,W004,W005,W006,W007,W008,W009,...,W231,W232,W233,W234,W235,W236,W237,W238,W239,W240
供应商ID,,,,,,,,,,,,,,,,,,,,,
S001,B,0,0,0,43,0,0,0,0,1,...,1,0,0,1,0,1,0,0,0,0
S002,A,1,1,0,1,0,0,0,0,0,...,0,0,1,0,0,0,1,1,0,1
S003,C,7,1,0,0,0,1,4,50,130,...,10,3,50,20,20,7,10,9,20,10
S004,B,0,1,1,100,0,85,0,0,0,...,1,0,1,0,0,0,0,1,0,0
S005,A,30,60,60,60,70,70,60,70,70,...,80,70,70,70,70,70,70,70,70,80


,材料分类,W001,W002,W003,W004,W005,W006,W007,W008,W009,...,W231,W232,W233,W234,W235,W236,W237,W238,W239,W240
供应商ID,,,,,,,,,,,,,,,,,,,,,
S001,B,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,2,0,0,0,0
S002,A,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
S003,C,8,1,0,0,0,1,5,58,110,...,9,4,54,20,20,8,18,10,22,11
S004,B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
S005,A,37,62,60,65,76,76,65,71,72,...,84,78,77,79,78,75,74,74,76,81


<p style="font-size: 20px">
SUM列 是将ABC类材料转为了同一个单位,方便比较<br>
AVG列 是 SUM / 供应次数<br>
diffAvg 也是一样</p>

In [44]:
supply["supplyTimes"] = (supply.loc[:, 'W001': 'W240'] != 0).sum(axis=1)
supply["SUM"] = supply.loc[:, 'W001': 'W240'].sum(axis=1).astype('float64')
dic = {'A': 0.6, 'B': 0.66, 'C': 0.72}
    
supply["AVG"] = supply["SUM"] / supply["supplyTimes"]

diff = supply.loc[:, 'W001': 'W240'] - order.loc[:, 'W001': 'W240']
supply["diffAvg"] = diff.sum(axis=1) / supply['supplyTimes']

for key, val in dic.items():
    supply.loc[supply['材料分类'] == key, "SUM": "diffAvg"] /= val

diffStd = []
for i in diff.values:
    diffStd.append(i[i != 0].std())
supply["diffStd"] = diffStd

supply

,材料分类,W001,W002,W003,W004,W005,W006,W007,W008,W009,...,W236,W237,W238,W239,W240,supplyTimes,SUM,AVG,diffAvg,diffStd
供应商ID,,,,,,,,,,,,,,,,,,,,,
S001,B,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,25,74.242424,2.969697,-11.030303,7.590148
S002,A,1,0,0,1,0,0,0,0,0,...,0,0,0,0,1,71,455.000000,6.408451,-0.845070,8.002821
S003,C,8,1,0,0,0,1,5,58,110,...,8,18,10,22,11,191,18247.222222,95.535195,-8.296975,54.557428
S004,B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,33,96.969697,2.938476,-29.797980,23.851153
S005,A,37,62,60,65,76,76,65,71,72,...,75,74,74,76,81,107,11520.000000,107.663551,5.825545,4.442075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S398,C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,16,56.944444,3.559028,-13.715278,16.816914
S399,B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,30,98.484848,3.282828,-11.161616,26.755169
S400,B,0,0,0,0,0,0,0,0,4,...,0,0,0,0,0,13,80.303030,6.177156,-87.762238,31.445255


<p style="font-size: 20px">
差值有些太离谱了，去除一些箱线图内限以外的离群值<br>
剔除了402-347个数据
</p>

In [45]:
desc = supply['diffAvg'].describe()
Q1 = desc["25%"]
Q3 = desc["75%"]
drop_index = supply[supply['diffAvg'] < Q1 - 1.5*(Q3-Q1)].index
supply.drop(index=drop_index, inplace=True)

supply

,材料分类,W001,W002,W003,W004,W005,W006,W007,W008,W009,...,W236,W237,W238,W239,W240,supplyTimes,SUM,AVG,diffAvg,diffStd
供应商ID,,,,,,,,,,,,,,,,,,,,,
S001,B,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,25,74.242424,2.969697,-11.030303,7.590148
S002,A,1,0,0,1,0,0,0,0,0,...,0,0,0,0,1,71,455.000000,6.408451,-0.845070,8.002821
S003,C,8,1,0,0,0,1,5,58,110,...,8,18,10,22,11,191,18247.222222,95.535195,-8.296975,54.557428
S004,B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,33,96.969697,2.938476,-29.797980,23.851153
S005,A,37,62,60,65,76,76,65,71,72,...,75,74,74,76,81,107,11520.000000,107.663551,5.825545,4.442075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S397,B,0,1,0,0,0,0,0,0,1,...,1,0,0,0,0,52,78.787879,1.515152,-0.233100,0.000000
S398,C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,16,56.944444,3.559028,-13.715278,16.816914
S399,B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,30,98.484848,3.282828,-11.161616,26.755169


<p style="font-size: 20px">topsis 评价指标：</p>
<ul>
    <li>supplyTimes 供应次数</li>
    <li>AVG 订单的平均供应量</li>
    <li>diffAvg 供应量与订单量差值的平均值</li>
    <li>diffStd 供应量与订单量差值的标准差</li>
</ul>

In [46]:
def topsis(data, weight=None):
    data = np.asarray(data)
    data = (data - data.min(axis=0)) / (data.max(axis=0) - data.min(axis=0) + 1e-4)
    
    if weight is None:
        p = data / np.square(data).sum(axis=0)
        e = np.zeros_like(p)
        e[p != 0] = p[p != 0] / np.log(p[p != 0])
        ent = -(1 / np.log(data.shape[0])) * e.sum(axis=0)
        weight = (1 - ent) / (1 - ent).sum()
    else:
        weight = np.asarray(weight).ravel()
    max_dist = np.sqrt((np.square(data - data.max(axis=0)) * weight).sum(axis=1))
    min_dist = np.sqrt((np.square(data - data.min(axis=0)) * weight).sum(axis=1))
    score = min_dist / (max_dist + min_dist)
    
    return (score - score.min()) / (score.max() - score.min())

data = supply[["supplyTimes", "AVG", "diffAvg", "diffStd"]]
data.loc[:, "diffStd"] *= -1
x = data.loc[:, "diffAvg"] 
data.loc[:, "diffAvg"] = 1 - (x - 0).abs() / (x - 0).abs().max()
score = topsis(data)
supply["SCORE"] = score
supply.sort_values(by='SCORE', ascending=False, inplace=True)
#supply.to_excel("../data/Q1.xlsx")
supply.head()

,材料分类,W001,W002,W003,W004,W005,W006,W007,W008,W009,...,W237,W238,W239,W240,supplyTimes,SUM,AVG,diffAvg,diffStd,SCORE
供应商ID,,,,,,,,,,,,,,,,,,,,,
S282,A,2,162,116,108,139,237,367,470,478,...,1412,1271,1036,708,240,282233.333333,1175.972222,5.618056,11.104766,1.000000
S275,A,395,525,577,546,643,651,487,618,697,...,744,532,786,779,240,264255.000000,1101.062500,2.798611,14.442292,0.983365
S329,A,495,494,612,573,681,486,628,720,606,...,676,632,553,730,240,260863.333333,1086.930556,1.166667,17.201788,0.980087
S340,B,42,609,734,761,791,732,584,759,878,...,811,742,779,961,240,259736.363636,1082.234848,-3.623737,49.970017,0.953298
S268,C,509,516,522,521,569,574,433,481,498,...,495,507,531,535,240,180258.333333,751.076389,2.407407,13.197697,0.905750


<p style="font-size: 20px">
将损耗率的平均值作为估计值
</p>

In [47]:
trans = pd.read_excel("../附件2 近5年8家转运商的相关数据.xlsx", index_col=0)
avgLossRate = np.mean(trans[trans != 0]) * 0.01
avgLossRate

np.float64(0.013334812630117977)

<p style="font-size: 20px">
使用遗传算法来求解模型
</p>

In [48]:
from genetic import Genetic

supply_50 = supply.iloc[:50]

def feasible(ind):
    supplyAbility = supply_50["AVG"]
    # supplyAbility = supply_50.loc[:, 'W001': 'W240'].max(axis=1)
    return (ind * supplyAbility).sum() * (1 - avgLossRate) >= 2.82e4

def evaluate(ind):
    if feasible(ind):
        return -sum(ind) / (ind * supply_50["SCORE"]).sum(),
    return -100,

model = Genetic(evaluate, geneSize=supply_50.shape[0])
result = model.run()
print(np.asarray(result))

[0 1 1 0 0 1 1 1 0 1 0 1 0 1 1 0 1 1 1 0 0 1 1 0 1 1 1 0 1 0 1 1 1 0 1 0 1
 0 1 0 1 1 1 1 1 1 1 0 0 1]


In [49]:
supply_50.loc[np.asarray(result) == 1]

,材料分类,W001,W002,W003,W004,W005,W006,W007,W008,W009,...,W237,W238,W239,W240,supplyTimes,SUM,AVG,diffAvg,diffStd,SCORE
供应商ID,,,,,,,,,,,,,,,,,,,,,
S275,A,395,525,577,546,643,651,487,618,697,...,744,532,786,779,240,264255.000000,1101.062500,2.798611,14.442292,0.983365
S329,A,495,494,612,573,681,486,628,720,606,...,676,632,553,730,240,260863.333333,1086.930556,1.166667,17.201788,0.980087
S306,C,406,260,320,344,408,379,385,356,558,...,548,473,546,538,240,175133.333333,729.722222,2.407407,15.002286,0.900356
S131,B,2,539,657,628,620,663,426,661,699,...,684,608,599,660,240,208351.515152,868.131313,-11.799242,72.079155,0.874412
S352,A,86,86,80,159,129,167,249,282,235,...,490,480,483,501,240,148385.000000,618.270833,-1.868056,31.200510,0.868828
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S360,B,2,0,1,0,1,0,5,0,4,...,5,0,0,0,131,613.636364,4.684247,-0.323849,4.264068,0.673827
S263,C,3,0,5,0,1,0,1,0,4,...,2,0,2,0,132,593.055556,4.492845,-0.473485,6.022664,0.673687
S216,B,2,0,2,0,4,0,1,0,2,...,5,0,3,0,129,643.939394,4.991778,0.516796,1.000845,0.673217
